In [29]:
import pandas as pd
from itertools import product

In [30]:
df_original = pd.read_csv("sales_train_dates_added.csv")
df_original.columns

Index(['Unnamed: 0', 'date_x', 'date_block_num', 'shop_id', 'item_id',
       'item_price', 'item_cnt_day', 'date_y', 'day_name', 'day_of_week',
       'day_of_month', 'week', 'month_name', 'month_of_year',
       'month_year_name', 'month_year_name_sort', 'quarter',
       'quarter_year_name_sort', 'year', 'year_half', 'red_day_not_sun',
       'black_friday', 'Date_formatted'],
      dtype='object')

In [31]:
#BlackFriday, Holiday, Red Day
#month_of_year, quarter, red_day_not_sun, black_friday, , year_half

df_draft = df_original[["date_x", "date_block_num", "shop_id", "item_id", "item_cnt_day","month_of_year", "quarter", "red_day_not_sun", "black_friday", "year_half"]].copy()
df_draft = df_draft.groupby(["date_block_num", "item_id", "shop_id"], as_index=False).sum()
df_result = pd.DataFrame()
for i in range(34):
    df_date_block = df_draft[df_draft["date_block_num"] == i][["date_block_num", "item_id", "shop_id"]]
    df_crossed = pd.DataFrame(list(product([i], df_date_block["item_id"].unique(), df_date_block["shop_id"].unique())),
                               columns=["date_block_num", "item_id", "shop_id"])
    df_result = pd.concat([df_result, df_crossed], ignore_index=True)
df_result = pd.merge(df_result, df_draft, on=["date_block_num", "item_id", "shop_id"], how="left")
df_result["item_cnt_day"].fillna(0, inplace=True)
df_result

,date_block_num,item_id,shop_id,date_x,item_cnt_day,month_of_year,quarter,red_day_not_sun,black_friday,year_half
0,0,19,25,12.01.2013,1.0,1.0,1.0,0.0,0.0,1.0
1,0,19,1,NaN,0.0,NaN,NaN,NaN,NaN,NaN
2,0,19,2,NaN,0.0,NaN,NaN,NaN,NaN,NaN
3,0,19,10,NaN,0.0,NaN,NaN,NaN,NaN,NaN
4,0,19,19,NaN,0.0,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
10913845,33,22167,34,NaN,0.0,NaN,NaN,NaN,NaN,NaN
10913846,33,22167,41,NaN,0.0,NaN,NaN,NaN,NaN,NaN
10913847,33,22167,48,NaN,0.0,NaN,NaN,NaN,NaN,NaN
10913848,33,22167,9,NaN,0.0,NaN,NaN,NaN,NaN,NaN


In [32]:
# import generate_mqy as gm
# Funker ikke, må fikses
# df_result['month_of_year'] = df_result['month_of_year'].apply(gm.int_to_month_year[2])
# df_result

TypeError: int_to_month_year() takes 1 positional argument but 2 were given

In [ ]:
df_sum = df_result.groupby(["date_block_num", "item_id", "shop_id"], as_index=False)["item_cnt_day"].sum()

In [ ]:
df_lag = df_sum.copy()
df_lag["lag_1"] = df_lag["date_block_num"] + 1
df_lag.isna().any()

In [ ]:
df_lag = pd.merge(df_sum, df_lag, left_on=["date_block_num", "item_id", "shop_id"], right_on=["lag_1", "item_id", "shop_id"], how="left")
df_lag = df_lag.sort_values("item_cnt_day_x", ascending=False)
df_lag = df_lag.drop(columns=["date_block_num_y", "lag_1"])
df_lag["item_cnt_day_y"].fillna(0, inplace=True)

In [ ]:
df_lag = df_lag.rename(columns={"item_cnt_day_x": "item_cnt_month", "item_cnt_day_y": "item_cnt_last_month"})
df_lag

In [ ]:
df_lag_12 = df_sum.copy()
df_lag_12["lag_12"] = df_lag_12["date_block_num"]+12
df_lag_12

In [ ]:
df_lag = pd.merge(df_lag, df_lag_12, left_on=["date_block_num_x", "item_id", "shop_id"], right_on=["lag_12", "item_id", "shop_id"], how="left")

In [ ]:
df_lag = df_lag.drop(columns=["date_block_num", "lag_12"])
df_lag = df_lag.rename(columns={"item_cnt_day": "item_cnt_last_year"})
df_lag


In [ ]:
df_lag["item_cnt_last_year"].fillna(0, inplace=True)
df_lag